In [1]:
import node
import numpy as np

Works on CPU


In [2]:
from tqdm import tqdm_notebook as tqdm

In [3]:
class LeNet(node.Network):
    
    def __init__(self, num_in_ch):
        self.layers = [
            # Block 1
            node.Convolution2D(num_in_ch, 16, 7, 1, 0),
            node.MaxPooling2D(2, 2),
            
            # Block 2
            node.Convolution2D(16, 30, 3, 1, 0),
            node.MaxPooling2D(2, 2),
            
            # FC
            node.Linear(480, 150),
            node.Linear(150, 10)
        ]
        
    def __call__(self, input):
        hidden = input
        
        # Block 1 
        # Output: 16 x 11 x 11
        hidden = self.layers[0](hidden)
        hidden = hidden.relu()
        hidden = self.layers[1](hidden)
        
        # Block 2
        # Output: 30 x 4 x 4
        hidden = self.layers[2](hidden)
        hidden = self.layers[3](hidden)
        hidden = hidden.tanh()
        
        # FC 
        hidden = hidden.reshape(input.value.shape[0], -1)
        hidden = self.layers[4](hidden)
        hidden = hidden.relu()
        hidden = self.layers[5](hidden)
        
        return hidden
    
classifier = LeNet(1)
optimizer = node.Adam(classifier.get_parameters(), 0.001)

Has 78810 parameters


In [4]:
datasets = [node.MNIST(train=True), 
            node.MNIST(train=False)]

In [5]:
dataloaders = [node.DataLoader(datasets[0], 100),
               node.DataLoader(datasets[1], 100)]

In [6]:
def train(input, target):
    prediction = classifier(input / 255)
    output = prediction.softmax_with_binary_cross_entropy(target)
    
    optimizer.clear()
    output.backward()
    optimizer.update()
    
    return output.numpy()

In [7]:
def evaluate(input, target):
    with node.zero_grad():
        prediction = classifier(input / 255)
        output = prediction.softmax_with_binary_cross_entropy(target)
        
    return output.numpy()

In [8]:
for epoch in range(10):
    # Train Loss, Test Loss, Accuracy
    metrics = [0, 0, 0]

    for input, target in tqdm(dataloaders[0]):
        metrics[0] += train(input, target)
        
    for input, target in dataloaders[1]:
        metrics[1] += evaluate(input, target)
        
    metrics[0] /= len(dataloaders[0])
    metrics[1] /= len(dataloaders[1])
    print("epoch {0:2}, train {1:.4f}, test {2:.4f}".format(epoch, *metrics))

KeyboardInterrupt: 